In [5]:
import numpy as np  # 넘파이 라이브러리를 가져옵니다. 수치 계산에 사용됩니다.
import pandas as pd # 판다스 라이브러리를 가져옵니다. 데이터 조작 및 분석에 사용됩니다.
from sklearn.preprocessing import StandardScaler # 데이터 전처리를 위해 StandardScaler를 가져옵니다. 특성 스케일링에 사용됩니다.

In [32]:
# 오류역전파와 시그모이드 활성화 함수를 사용하는 신경망 클래스
class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # 클래스 초기화 시 가중치를 무작위로 초기화합니다.
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)  # 입력층에서 은닉층으로 가는 가중치
        self.weights_hidden_output = np.random.randn(hidden_size, output_size) # 은닉층에서 출력층으로 가는 가중치

    def sigmoid(self, x):
        # 시그모이드 활성화 함수: 신경망의 출력을 0과 1 사이로 제한합니다.
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        # 시그모이드 함수의 미분: 역전파 시 사용됩니다.
        return x * (1 - x)

    def train(self, inputs, labels, epochs, learning_rate):
        # 신경망을 훈련하는 함수
        for epoch in range(epochs):
            # 순전파 과정
            # 입력층에서 은닉층으로의 신호 계산
            hidden_input = np.dot(inputs, self.weights_input_hidden)
            # 은닉층의 출력 계산 (활성화 함수 적용)
            hidden_output = self.sigmoid(hidden_input)
    
            # 은닉층에서 출력층으로의 신호 계산
            final_input = np.dot(hidden_output, self.weights_hidden_output)
            # 출력층의 출력 계산 (활성화 함수 적용)
            final_output = self.sigmoid(final_input)
    
            # 오차 계산
            # 실제 레이블과 예측 값 간의 차이
            error = labels - final_output
            
            # 역전파 과정
            # 출력층의 오차 신호 계산
            d_final_output = error * self.sigmoid_derivative(final_output)
            # 은닉층의 오차 신호 계산
            error_hidden_layer = d_final_output.dot(self.weights_hidden_output.T)
            # 은닉층의 오차 신호에 대한 미분 값 계산
            d_hidden_layer = error_hidden_layer * self.sigmoid_derivative(hidden_output)
    
            # 가중치 업데이트
            # 은닉층에서 출력층으로 가는 가중치 업데이트
            self.weights_hidden_output += hidden_output.T.dot(d_final_output) * learning_rate
            # 입력층에서 은닉층으로 가는 가중치 업데이트
            self.weights_input_hidden += inputs.T.dot(d_hidden_layer) * learning_rate
    
            # 에포크마다 손실을 출력
            if epoch % 1000 == 0:
                loss = np.mean(np.abs(error))
                print(f'Epoch {epoch}, Loss: {loss}')


    def predict(self, inputs):
        # 신경망을 사용하여 입력 데이터에 대한 예측을 수행하는 함수
        hidden_input = np.dot(inputs, self.weights_input_hidden)
        hidden_output = self.sigmoid(hidden_input)

        final_input = np.dot(hidden_output, self.weights_hidden_output)
        final_output = self.sigmoid(final_input)

        return final_output


In [33]:
# 훈련 및 테스트 데이터 파일 경로 설정
train_data_path = 'training.dat'  # 훈련 데이터 파일의 경로
test_data_path = 'testing.dat'    # 테스트 데이터 파일의 경로

# Pandas를 사용하여 훈련 및 테스트 데이터 파일 로드
# 파일에 헤더가 없고 공백으로 구분된 데이터로 가정
train_data = pd.read_csv(train_data_path, header=None, delim_whitespace=True)  # 훈련 데이터 로드
test_data = pd.read_csv(test_data_path, header=None, delim_whitespace=True)    # 테스트 데이터 로드

# 데이터의 처음 몇 줄을 출력하여 구조 확인
# 이는 데이터에 어떤 특성이 있는지, 어떤 형태의 데이터인지 이해하기 위함
train_data.head(), test_data.head()


(     0    1    2    3
 0  6.3  3.3  6.0  2.5
 1  5.8  2.7  5.1  1.9
 2  7.1  3.0  5.9  2.1
 3  6.3  2.9  5.6  1.8
 4  6.5  3.0  5.8  2.2,
      0    1    2    3
 0  7.2  3.2  6.0  1.8
 1  6.2  2.8  4.8  1.8
 2  6.1  3.0  4.9  1.8
 3  6.4  2.8  5.6  2.1
 4  7.2  3.0  5.8  1.6)

In [34]:
# 데이터 로드
train_data = pd.read_csv('training.dat', header=None, delim_whitespace=True)
test_data = pd.read_csv('testing.dat', header=None, delim_whitespace=True)

# 특성과 레이블 분리
X_train = train_data.iloc[:, :-1].values  # 훈련 데이터에서 마지막 열을 제외한 모든 열을 특성으로 설정
y_train = train_data.iloc[:, -1].values   # 훈련 데이터의 마지막 열을 레이블로 설정
X_test = test_data.iloc[:, :-1].values    # 테스트 데이터에서 마지막 열을 제외한 모든 열을 특성으로 설정
y_test = test_data.iloc[:, -1].values     # 테스트 데이터의 마지막 열을 레이블로 설정

# 데이터 표준화
scaler = StandardScaler()                 # StandardScaler 인스턴스 생성
X_train = scaler.fit_transform(X_train)   # 훈련 데이터를 표준화 (평균 0, 표준편차 1)
X_test = scaler.transform(X_test)         # 테스트 데이터를 같은 스케일로 표준화 (훈련 데이터 기준)


In [43]:
# 신경망 인스턴스 생성
# SimpleNeuralNetwork 클래스의 인스턴스를 생성합니다.
# input_size는 입력층의 노드 수, hidden_size는 은닉층의 노드 수, output_size는 출력층의 노드 수를 의미합니다.
nn = SimpleNeuralNetwork(input_size=X_train_scaled.shape[1], hidden_size=4, output_size=1)

# 신경망 학습
# 훈련 데이터의 레이블(y_train)을 신경망의 출력 형태에 맞게 조정합니다.
# 여기서는 y_train을 (데이터 개수, 1)의 형태로 변경하여 각 데이터 포인트에 대한 하나의 출력값이 있도록 합니다.
y_train = y_train.reshape(-1, 1)  # y_train을 (75, 1) 형태로 조정

# train 메서드를 사용하여 신경망 모델을 학습시킵니다.
# X_train_scaled는 표준화된 훈련 데이터의 특성,
# y_train은 훈련 데이터의 레이블,
# epochs는 학습을 진행할 총 반복 횟수,
# learning_rate는 학습률(가중치 조정 시 적용되는 비율)을 의미합니다.
nn.train(X_train_scaled, y_train, epochs=10000, learning_rate=0.1)


Epoch 0, Loss: 0.8927790327799806
Epoch 1000, Loss: 0.48268799142954083
Epoch 2000, Loss: 0.4826322707368309
Epoch 3000, Loss: 0.4826125420789385
Epoch 4000, Loss: 0.4826030034180308
Epoch 5000, Loss: 0.48259893450632935
Epoch 6000, Loss: 0.482597425863056
Epoch 7000, Loss: 0.4825967611223472
Epoch 8000, Loss: 0.4825956880436316
Epoch 9000, Loss: 0.4825930780861762


In [44]:
# 테스트 데이터에 대한 예측 수행
# 신경망 모델(nn)의 'predict' 메서드를 사용하여 X_test_scaled 데이터에 대한 예측을 수행합니다.
y_pred = nn.predict(X_test_scaled)

# 신경망의 출력을 이진 레이블로 변환
# 신경망은 확률 값을 출력하기 때문에, 이를 이진 레이블(0 또는 1)로 변환합니다.
# 0.5를 임계값으로 설정하여, 0.5보다 큰 값은 1로, 그렇지 않은 값은 0으로 변환합니다.
y_pred_classes = (y_pred > 0.5).astype(int)

# 실제 레이블과 예측 레이블을 비교하여 정확도 계산
# y_test는 테스트 데이터의 실제 레이블을 나타냅니다.
# y_pred_classes는 신경망이 예측한 레이블입니다.
# 이 두 배열을 비교하여 평균적으로 얼마나 일치하는지 계산하여 정확도를 도출합니다.
accuracy = np.mean(y_pred_classes.flatten() == y_test)
print("Accuracy:", accuracy)


Accuracy: 0.04
